# Creating a NATS input file from IFF data
## Step 1a: Read in IFF Data using PARA-ATM

In [1]:
from paraatm.io import iff, gnats, utils
from paraatm.io.iff import read_iff_file

iff_fname = 'IFF_SFO+ASDEX_20190511_080104_86221.csv'#'IFF_SFO_ASDEX_ABC456.csv'

iff_data =  read_iff_file(iff_fname,record_types=[2,3,4,8])

## Step 1b: Select flights within a given timeframe

In [2]:
import pandas as pd

time_interval = 1200

start_time = iff_data[2]['time'].iloc[0] 
end_time =  pd.to_datetime(start_time.value*10**(-9) + time_interval, unit='s')

iff_data[2] = iff_data[2].loc[iff_data[2].time <= end_time,:]
iff_data[3] = iff_data[3].loc[iff_data[3].time <= end_time,:]
iff_data[4] = iff_data[4].loc[iff_data[4].time <= end_time,:]
iff_data[8] = iff_data[8].loc[iff_data[8].time <= end_time,:]

callsigns = iff_data[3].callsign.unique()
cs = callsigns.tolist()

## Step 1c: Clean callsign list of UNKN, nan, 1200, and ground operations vehicles

In [3]:
if 'UNKN' in cs: cs.remove('UNKN') # remove 'UNKN'
cs = [x for x in cs if (str(x) != 'nan')] #remove nan
cs = [x for x in cs if not x.startswith('OPS')] #remove ground operation vehicles
if '1200' in cs: cs.remove('1200')

## Step 2: Start NATS Simulation in the background

In [4]:
from gnats_gate_to_gate import GateToGate
natsSim = GateToGate()

## Step 3: Create FlightPlanSelector object

In [5]:
from paraatm import fpgen
dirPath = natsSim.DIR_share
fpath = '/home/ghaikal/para-atm/paraatm/fpgen/TRX_07132005_noduplicates_crypted_SFO_clean'
f=fpgen.FlightPlanSelector(natsSim,fname=fpath)

## Step 4: Remove previously generated TRX and MFL files

In [6]:
#Filenames to write modified trx and mfl files to. Need full path because GNATS wrapper changes the directory to the GNATS directory.
import os

home_env = os.environ.get('HOME')
trx_dir = '/home/ghaikal/para-atm-collection/miscellaneous/gnats-fpgen'
results_dir = '/home/ghaikal/para-atm-collection/miscellaneous/gnats-fpgen'

trx_fname = '/iff_to_gnats_geo_SFO_demo'
mfl_file= trx_dir+trx_fname+'_mfl.trx'
trx_file = trx_dir+trx_fname+'.trx'
results_file = results_dir+trx_fname+'.csv'

if os.path.exists(trx_file):
    os.remove(trx_file)
    print('Previously generated trx file removed...')

if os.path.exists(mfl_file):
    os.remove(mfl_file)
    print('Previously generated mfl file removed...')

## Step 5 Generate NATS TRX and MFL files

In [7]:
import time
import numpy as np
import pandas as pd
from jpype import JPackage

from paraatm.fpgen import (get_departure_airport_from_iff,
                           get_arrival_airport_from_iff,
                           get_gate_from_iff,
                           get_rwy_from_iff,
                           check_if_flight_has_departed_from_iff,
                           check_if_flight_landing_from_iff,
                           get_gate_lat_lon_from_nats,
                           get_random_gate,
                           get_random_runway)

iffBaseAirport= "KSFO"
clsGeometry = JPackage('com').osi.util.Geometry
TRACK_TIME = time.time()

for i,callsign in enumerate(cs):
    flightInAir = check_if_flight_has_departed_from_iff(iff_data,callsign,natsSim,iffBaseAirport)
    flightLandingInData = check_if_flight_landing_from_iff(iff_data,callsign,natsSim,iffBaseAirport)
    
    arrivalAirport = ''
    departureAirport=''
    arrivalRwy = ''
    arrivalGate = ''
    departureRwy =''
    departureGate = ''
    result_generated = ''

    if flightInAir and flightLandingInData:
        #Get departure airport. If none is known in the IFF+ASDEX file (i.e., it is not the airport whose name is in the iff_fname), then set departure airport as closest airport to the first lat/lon in the dataset. In the future, would like to use IFF_USA to determine departureAirport in this case
        arrivalAirport = 'KSFO'
        departureAirport = get_departure_airport_from_iff(iff_data,callsign,natsSim,arrivalAirport=iffBaseAirport,flmap=f.flmap)
        arrivalGate= get_gate_from_iff(iff_data,callsign,natsSim,arrivalAirport)
        arrivalRwy= get_rwy_from_iff(iff_data,callsign,natsSim,arrivalAirport,arrival=True)

        result_generated = f.generate(4, departureAirport, arrivalAirport, "", arrivalGate, "", arrivalRwy)
        
        timestamp = iff_data[3].loc[iff_data[3].callsign==callsign,'time'].iloc[0]
        lat,lon = list(natsSim.airportInterface.getLocation(departureAirport))
        latstr = clsGeometry.convertLatLonDeg_to_degMinSecString(str(lat))
        lonstr = clsGeometry.convertLatLonDeg_to_degMinSecString(str(lon))
        elev = np.max([iff_data[3].loc[iff_data[3].callsign==callsign,'altitude'].iloc[0]/100.,100.])
        spd = iff_data[3].loc[iff_data[3].callsign==callsign,'tas'].iloc[0]
        hdg = iff_data[3].loc[iff_data[3].callsign==callsign,'heading'].iloc[0]
        aircraftType = 'B744'
        
    if  not flightInAir and not flightLandingInData:
        departureAirport = 'KSFO'
        arrivalAirport = get_arrival_airport_from_iff(iff_data,callsign,natsSim,departureAirport,f.flmap)
        departureGate = get_gate_from_iff(iff_data,callsign,natsSim,departureAirport)
        departureRwy = get_rwy_from_iff(iff_data,callsign,natsSim,departureAirport,arrival=False)

        result_generated = f.generate(5, departureAirport, arrivalAirport, departureGate, "", departureRwy, "")
        
        timestamp = iff_data[3].loc[iff_data[3].callsign==callsign,'time'].iloc[0]
        airportInstance = natsSim.airportInterface.select_airport(departureAirport)
        elev = airportInstance.getElevation()/100.0  
        lat,lon = get_gate_lat_lon_from_nats(natsSim,departureGate,departureAirport)
        latstr = clsGeometry.convertLatLonDeg_to_degMinSecString(str(lat))
        lonstr = clsGeometry.convertLatLonDeg_to_degMinSecString(str(lon))
        aircraftType = 'B744'
        spd = 0
        hdg = 28
    
    if result_generated:
        #TRACK_TIME = (timestamp-pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
        TRACK_TIME +=10
        track_string = '%s %s %.4f %.4f %d %.2f %d %s %s' %(callsign,aircraftType,float(latstr),float(lonstr),spd,elev,hdg,'ZOA','ZOA46')
        fp_route = result_generated[0]
        fp_validated = natsSim.aircraftInterface.validate_flight_plan_record(track_string,fp_route,330)
        print('Validated Flight Plan:',callsign)
        fp_validated = True
        if fp_validated:
            print(result_generated)
            with open(trx_file,'a') as trxFile:
                tstr = '%s %s' % ('TRACK',track_string)
                trxFile.write('%s %d' % ('TRACKTIME', TRACK_TIME) + '\n')
                trxFile.write( tstr + '\n')
                trxFile.write('    FP_ROUTE ' + fp_route + '\n\n')

            with open(mfl_file,'a') as mflFile:
                mflFile.write(callsign + ' ' + '330' + '\n')
        elif not fp_validated:
            print('This flight plan was not validated:\n', result_generated[0])
                                    

0.00920477398949693 6.0 13.0
No viable origin airport found for DAL2312. Returning random from FlightPlanSelector options.
Random Gate at KSFO: Gate_D_51A
Usable Runways for Arrival at KSFO: ['RW10L', 'RW10R', 'RW19L', 'RW19R', 'RW28L', 'RW28R']
Rwy at KSFO from IFF: RW28R
legWps: ('RW12-KLGB', 'RW12-KLGB-NO_WP_NAME') lastLatLonVal: (33.823594444444446, -118.15837777777779) 

New Distance:  103.65734555756329
OptimalSid:  ANAHM1
legWps: ('HAWWC', 'WROBN', 'ORIYL', 'MOOOS', 'IKAYE') lastLatLonVal: (33.915255555555554, -118.4141) 

New Distance:  87.72985525529673
OptimalSid:  HAWWC3
OptimalApproach I28R
First Approach Waypoint: ARCHI  LatLon: (37.49079722222222, -121.87554166666666) 

legWps: ('ALWYS', 'HEFLY', 'ARRTU', 'ADDMM', 'COGGR', 'BERKS', 'BERKS') lastLatLonVal: (37.86086111111111, -122.21167777777778) 

New Distance:  31.490687320750855
OptimalStar: ALWYS2
legWps: ('LOZIT', 'BDEGA', 'CORKK', 'BRIXX', 'BRIXX') lastLatLonVal: (37.617844444444444, -122.37452777777777) 

New Distan

No viable origin airport found for JBU1715. Returning random from FlightPlanSelector options.
Random Gate at KSFO: Gate_E_060
Usable Runways for Arrival at KSFO: ['RW10L', 'RW10R', 'RW19L', 'RW19R', 'RW28L', 'RW28R']
Rwy at KSFO from IFF: RW28R
legWps: ('PORTT', 'RIETA', 'SBJ', 'BIGGY') lastLatLonVal: (40.590805555555555, -74.29395) 

New Distance:  276.37800942620976
OptimalSid:  PORTT4
OptimalApproach I28R
First Approach Waypoint: ARCHI  LatLon: (37.49079722222222, -121.87554166666666) 

legWps: ('ALWYS', 'HEFLY', 'ARRTU', 'ADDMM', 'COGGR', 'BERKS', 'BERKS') lastLatLonVal: (37.86086111111111, -122.21167777777778) 

New Distance:  31.490687320750855
OptimalStar: ALWYS2
legWps: ('LOZIT', 'BDEGA', 'CORKK', 'BRIXX', 'BRIXX') lastLatLonVal: (37.617844444444444, -122.37452777777777) 

New Distance:  28.70718676105714
OptimalStar: BDEGA3
legWps: ('BSR', 'CARME', 'ANJEE', 'SKUNK', 'BOLDR', 'MENLO') lastLatLonVal: (37.463686111111116, -122.15365833333334) 

New Distance:  15.364332556171126
O

## Step 6: Simulate generated TRX and MFL files in NATS

The generated files can be simulated in NATS using the co-located Jupyter notebook <code>run_nats.ipynb</code> which can be found <a href=http://localhost:8888/notebooks/run_nats.ipynb>here</a>.J